<a href="https://colab.research.google.com/github/Vardhini723/Fmml-labs/blob/main/Copy_of_FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

NameError: name 'dataset' is not defined

The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4147
Number of train samples: 16493
Percent of test data: 20.092054263565892 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

NameError: name 'traindata' is not defined

For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

NameError: name 'traindata' is not defined

Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

NameError: name 'alltraindata' is not defined

You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

NameError: name 'alltraindata' is not defined

### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
Ans:-
The accuracy of the validation set can be affected by the size of the validation set in the following ways:

Increasing the Percentage of the Validation Set:

Pros: A larger validation set can provide a more stable and representative estimate of your model's performance because it includes more data points. This can lead to a more accurate assessment of the model's generalization ability.
Cons: With a larger validation set, there will be less data left for training the model. This might lead to less effective training and potentially lower performance of the model itself.
Reducing the Percentage of the Validation Set:

Pros: A smaller validation set allows more data to be used for training, which can improve the model’s ability to learn and generalize from the training data.
Cons: A smaller validation set might not be as representative of the overall data distribution, leading to a less reliable estimate of the model's performance. The accuracy on the validation set might become more variable and less indicative of the model's performance on unseen data.
In summary, increasing the size of the validation set can lead to a more reliable estimate of model performance, but at the cost of less training data. Conversely, reducing the validation set size gives more data for training but can lead to less reliable performance estimates.

2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?

Ans:- The size of the train and validation sets can significantly impact the ability to predict the accuracy on the test set. Here's how:

Train Set Size:

Larger Train Set: Generally, a larger training set helps the model learn more about the data distribution, leading to better generalization. This can make the validation set’s accuracy more reliable as a predictor of test set accuracy.
Smaller Train Set: With a smaller training set, the model might not learn effectively, leading to poorer generalization. This can cause the validation set accuracy to be less indicative of the test set performance.
Validation Set Size:

Larger Validation Set: A larger validation set provides a more stable estimate of the model's performance and reduces variance in the accuracy estimate. This means it’s more likely to provide a reliable prediction of test set performance.
Smaller Validation Set: A smaller validation set might be less representative of the overall data distribution, leading to higher variance in performance estimates. This can make it harder to predict how well the model will perform on the test set.
Key Points:

Balance is Crucial: Both the training and validation sets need to be large enough to provide accurate insights. If either set is too small, the model’s performance metrics might not generalize well to unseen data.
Trade-offs: Increasing the validation set size reduces the amount of data available for training, potentially affecting model performance. Thus, finding an optimal balance is important.
Cross-Validation: Techniques like k-fold cross-validation can help address these issues by using multiple validation sets and aggregating results to get a more reliable estimate of model performance.
In summary, having appropriately sized training and validation sets improves the accuracy and reliability of predicting test set performance.

3.W hat do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

QUESTION **1**     If you increase the percentage of the validation set, the accuracy metric becomes more reliable but you have less data to train the model, which could slightly affect its performance. If you reduce the percentage of the validation set, you get more data for training, but the accuracy metric becomes less reliable and might fluctuate more.

QUESTION **2** A larger validation set gives a more accurate prediction of test set performance, but leaves less data for training, which might slightly hurt the model. A smaller validation set gives more data for training, but its prediction of test set performance may be less reliable.

QUESTION **3**  A good percentage to reserve for the validation set is usually around 20%. This balances having enough data for reliable validation while still keeping most of the data for training.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [1]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

NameError: name 'NN' is not defined

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


QUESTION **1**
Yes, averaging the validation accuracy across multiple splits (like in cross-validation) gives more consistent and reliable results because it reduces the impact of any single, possibly unrepresentative, split.

QUESTION **2** Yes, averaging validation accuracy across multiple splits gives a more accurate estimate of test accuracy because it reflects the model's performance on different subsets of the data.

QUESTION **3**  Yes, with more iterations, you get a better estimate because the results are averaged over more trials, reducing the impact of random variations and leading to a more accurate and stable estimate.

QUESTION **4** Yes, by increasing the iterations (like using cross-validation), you can get more reliable estimates even with a very small train or validation dataset, as it allows the model to learn and validate on different subsets of the data multiple times.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Load dataset
data = load_iris()
X, y = data.data, data.target

def evaluate_knn(k, X, y, num_splits):
    kf = KFold(n_splits=num_splits, shuffle=True, random_state=42)
    accuracies = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        clf = KNeighborsClassifier(n_neighbors=k)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))

    return np.mean(accuracies)

# Define splits to test
split_numbers = [2, 5, 10]  # Different numbers of splits
ks = [1, 3]  # k values for 1-NN and 3-NN

# Results storage
results = {k: {} for k in ks}

for k in ks:
    for num_splits in split_numbers:
        accuracy = evaluate_knn(k, X, y, num_splits)
        results[k][num_splits] = accuracy

# Display results
for k in ks:
    print(f"Results for {k}-Nearest Neighbors:")
    for num_splits in split_numbers:
        print(f"  {num_splits} splits: {results[k][num_splits]:.4f} accuracy")

Results for 1-Nearest Neighbors:
  2 splits: 0.9533 accuracy
  5 splits: 0.9600 accuracy
  10 splits: 0.9600 accuracy
Results for 3-Nearest Neighbors:
  2 splits: 0.9600 accuracy
  5 splits: 0.9667 accuracy
  10 splits: 0.9667 accuracy
